<a href="https://colab.research.google.com/github/chags1313/graphs/blob/main/Train_and_Test_ACC_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 Train and Test ACC ML: An example machine learning script for human activity recognition 🤾‍♀️

Install pycaret amd shap
- Pycaret is a machine learning library
- For more information, check out the documentation https://pycaret.readthedocs.io/en/latest/
- Shap is a model explanation library
- **When installation finishes, watch for a button called 'restart runtime'**
- **Click this button then proceed to run next cells**

In [ ]:
!pip install shap
!pip install pycaret

# Data Prep

Import libraries

In [ ]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import plotly.express as px
from google.colab import files
import io
import shap
import logging
import sys
logging.disable(sys.maxsize)
print("Imported libraries - you may proceed to the next cell")

Read in your training data file 
- This should be one of the two experiments you performed with 3 different activities
- This file will be used to train a model to identify the activities you performed from the accelerometer data

In [ ]:
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0].strip(""))
df.describe()

Visualize raw acceleration data by three axes over time

In [ ]:
px.line(df,x='Time (s)',y = ['Acceleration x (m/s^2)', 'Acceleration y (m/s^2)', 'Acceleration z (m/s^2)'])

Visualize absolute acceleration (averge of all axes) over time

In [ ]:
px.line(df, x= 'Time (s)', y = ['Absolute acceleration (m/s^2)'])

# Activity Annotation

Answer the prompts below to identify when your activities took place
- The following code will provide an input 
- Fill in the input with the correct information
- Your activity name should match the activities you performed
- Your activities start and end times should match the time in seconds as presented in the graphs above or match times in seconds that you noted while performing
- **Run the cell when all information is correct**

***Activity 1***

In [ ]:
Activity_1 = "Sitting" #@param {type:"string"}
Activity_1_Start_Time = 2 #@param {type:"number"}
Activity_1_Finish_Time = 142 #@param {type:"number"}

***Activity 2***

In [ ]:
Activity_2 = "Standing" #@param {type:"string"}
Activity_2_Start_Time = 150 #@param {type:"number"}
Activity_2_Finish_Time = 210 #@param {type:"number"}

***Activity 3***

In [ ]:
Activity_3 = "Walking" #@param {type:"string"}
Activity_3_Start_Time = 215 #@param {type:"number"}
Activity_3_Finish_Time = 350 #@param {type:"number"}

In [ ]:
df['class'] = 'NAN'
df['class'] = np.where(df['Time (s)'].between(Activity_1_Start_Time,Activity_1_Finish_Time), Activity_1, 0)
df['class'] = np.where(df['Time (s)'].between(Activity_2_Start_Time,Activity_2_Finish_Time), Activity_2, df['class'])
df['class'] = np.where(df['Time (s)'].between(Activity_3_Start_Time,Activity_3_Finish_Time), Activity_3, df['class'])
df = df[df['class'] != 'NAN']
df = df[df['class'] != '0']
px.scatter(df,x = 'Time (s)', y = 'class', color = 'class')

Plot absolute acceleration over time colored by activity

In [ ]:
px.scatter(df, x = 'Time (s)', y = 'Absolute acceleration (m/s^2)', color = 'class')

View the distribution of absolute acceleration by each activity 

In [ ]:
px.histogram(df,x='Absolute acceleration (m/s^2)', color = 'class')

Display features in relation to each activity

In [ ]:
pc_df = df.copy()
classes = df['class'].unique()
pc_df['class'] = pc_df['class'].replace(classes[0], 0)
pc_df['class'] = pc_df['class'].replace(classes[1], 1)
pc_df['class'] = pc_df['class'].replace(classes[2], 2)
print("Classes: " + " //CLASS 0 = " + classes[0] + " //CLASS 1 = " + classes[1] + " //CLASS 2 = " + classes[2])
pc = px.parallel_coordinates(pc_df.drop(['Time (s)'], axis = 1), 
                             color = 'class',title='How do our features [acceleration x,y,z] relate to your classes?', 
                             width=1300)
pc.show()

# Model Training

For more advanced machine learning data prep, check out the documentation:
https://pycaret.readthedocs.io/en/latest/api/classification.html

Steps for model training
1. Setup up data
2. Compare models and performance metrics
3. Evaluate model
4. Apply model to new, unseen data

In [ ]:
exp = setup(df.drop(['Time (s)'], axis = 1), target = 'class', silent=True, profile = True)

Assess multiple models to understand which perform the best with our data and saving the top model

In [ ]:
best = compare_models()

In [ ]:
print(best)

Visualize the performance of the model

In [ ]:
evaluate_model(best)

# Model Testing
Apply your model to unseen data
- Add your 2nd activity file
- This file should include the same three activities but may include different order or different timimg

In [ ]:
TEST_FILE = files.upload()
TEST = pd.read_csv('Raw Data (1).csv')
TEST

In [ ]:
predictions = predict_model(best, data = TEST)
predictions

In [ ]:
import matplotlib
shap.initjs()
Prediction_Second = 0 #@param {type:"slider", min:0, max:1000, step:1}
Your_Name = "Cole" #@param {type:"string"}
title_str = Your_Name + "'s model predictions on test data"
second_in_predictions = predictions[predictions['Time (s)'] > float(Prediction_Second)-0.1]
second_in_predictions = second_in_predictions[second_in_predictions['Time (s)'] < float(Prediction_Second)+0.1]
Prediction = second_in_predictions['Label'].iloc[0]
lab = str(Your_Name) + "'s Model Predicted " + str(Prediction) + " at " + str(Prediction_Second) + " seconds"
fig = px.scatter(predictions, 
                 x = 'Time (s)', 
                 y = 'Absolute acceleration (m/s^2)', 
                 color = 'Label', 
                 title = lab,
                 color_discrete_sequence=["#F1948A", "#48C9B0", "#85C1E9"])
fig.add_vline(x=Prediction_Second , line_dash="dot",
              annotation_text="Time of Prediciton Analyzed - the variables listed below identify what influenced this prediciton", 
              annotation_position="top right",
              annotation_font_size=10,
              annotation_font_color="purple"
             )
fig.show()
if Prediction == predictions['Label'].unique()[0]:
  cr1 = "#FF0000"
  cr2 = "#F1948A"
if Prediction == predictions['Label'].unique()[1]:
  cr1 = "#48C9B0"
  cr2 = '#186A3B'
if Prediction == predictions['Label'].unique()[2]:
  cr1 = "#0000FF"
  cr2 = "#85C1E9"
explainer = shap.KernelExplainer(model = best.predict_proba, data = get_config('X_train'), link = "identity")
shap_value_single = explainer.shap_values(X = predictions.drop(['Label', 'Score', 'Time (s)'], axis=1).iloc[second_in_predictions.index[0]], nsamples = 10)
shap.force_plot(base_value = explainer.expected_value[0],
                shap_values = shap_value_single[0],
                features = predictions.drop(['Label', 'Score', 'Time (s)'], axis=1).iloc[second_in_predictions.index[0]],
                 plot_cmap=[cr1,cr2]
                )
